<a href="https://colab.research.google.com/github/rymarinelli/Fire-Fly/blob/main/Distributed_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark 
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
import pyspark.sql.functions as F 

     |████████████████████████████████| 204.2MB 55kB/s 
     |████████████████████████████████| 204kB 42.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=51df02c04883ec393322640812cce42030a93d1204948e16f3e13fa02547dc75
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
conf = SparkConf().setMaster("local").setAppName("Distributed_Machine_Learning")
conf.set("spark.scheduler.mode", "FAIR")
conf.set('spark.task.cpus', '1')
sc = SparkContext(conf = conf)
sqlContext = SQLContext(sc)

In [ ]:
import keras 
import tensorflow as tf

In [ ]:
!pip install horovod[tensorflow,keras,spark]

     |████████████████████████████████| 3.2MB 8.4MB/s 
     |████████████████████████████████| 286kB 42.4MB/s 
     |████████████████████████████████| 20.7MB 1.4MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
  Created wheel for horovod: filename=horovod-0.21.1-cp36-cp36m-linux_x86_64.whl size=20679917 sha256=084cfa6238fc8874bfab446314d0e4c4ffe3218c71bff2bea207e2371d378e5e
  Stored in directory: /root/.cache/pip/wheels/a1/91/85/8e868c735394d482d578639f2dc392ae7f69e11a58a95d6108
Successfully built horovod
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
#import horovod.keras as hvd
from tensorflow.python.keras import backend as K
import math
import os
from tensorflow.keras import layers
import horovod.spark.keras as hvd
import numpy as np


In [ ]:
#config =  tf.compat.v1.ConfigProto()
#config.gpu_options.allow_growth = True
#config.gpu_options.visible_device_list = str(hvd.local_rank())
#K.set_session(tf.compat.v1.Session(config=config))

In [ ]:
!pip install PyDrive

#Import modules
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Get the Shareable link
#Ex link : https://drive.google.com/file/d/1c7Ffo1Go1dtUpKcSWxdbdVyW4dfhEoUp/view?usp=sharing
# Get the id from the link 1JYdxKasFKNfVqx2Iklj7iv_bwPanmi4j
downloaded = drive.CreateFile({'id':"1JYdxKasFKNfVqx2Iklj7iv_bwPanmi4j"})   
# Remember at least what was the file extension
downloaded.GetContentFile('fire_data.tar.gz')  

In [ ]:
!tar xvf fire_data.tar.gz
!mv Codes/datasets/tmp/* fire-light/Codes/datasets/

Codes/datasets/tmp/fire/
Codes/datasets/tmp/fire/20200904_090428_038.jpg
Codes/datasets/tmp/fire/20200904_090428_026.jpg
Codes/datasets/tmp/fire/20200904_090428_091.jpg
Codes/datasets/tmp/fire/20200724_175212_066.jpg
Codes/datasets/tmp/fire/20200724_175212_031.jpg
Codes/datasets/tmp/fire/20200904_090428_060.jpg
Codes/datasets/tmp/fire/20200724_175212_073.jpg
Codes/datasets/tmp/fire/20200904_090428_023.jpg
Codes/datasets/tmp/fire/20200724_175212_071.jpg
Codes/datasets/tmp/fire/20200724_175212_065.jpg
Codes/datasets/tmp/fire/20200904_090428_020.jpg
Codes/datasets/tmp/fire/20200904_090428_005.jpg
Codes/datasets/tmp/fire/20200904_090428_079.jpg
Codes/datasets/tmp/fire/20200724_175212_004.jpg
Codes/datasets/tmp/fire/20200904_090428_049.jpg
Codes/datasets/tmp/fire/20200904_090428_034.jpg
Codes/datasets/tmp/fire/20200904_090428_068.jpg
Codes/datasets/tmp/fire/20200904_090428_031.jpg
Codes/datasets/tmp/fire/20200724_175212_015.jpg
Codes/datasets/tmp/fire/20200904_090428_053.jpg
Codes/datasets/

In [ ]:
dir_fire = '/content/Codes/datasets/tmp/fire'
dir_no_fire = '/content/Codes/datasets/tmp/nofire'

In [ ]:
fire = len([name for name in os.listdir(dir_fire) if os.path.isfile(os.path.join(dir_fire, name))])
no_fire = len([name for name in os.listdir(dir_no_fire) if os.path.isfile(os.path.join(dir_no_fire, name))])
total = fire + no_fire
weight_for_fire = (1 / fire) * total / 2.0
weight_for_no_fire = (1 / no_fire) * total / 2.0
print("Weight for class fire : {:.2f}".format(weight_for_fire))
print("Weight for class No_fire : {:.2f}".format(weight_for_no_fire))


Weight for class fire : 0.77
Weight for class No_fire : 1.42


In [ ]:
new_size = {'width': 256, 'height': 256}
Config_classification = {"batch_size": 32, 'Save_Model': True, 'Epochs': 40, "TrainingPlot": True}
epochs = Config_classification.get('Epochs')
image_size = (new_size.get('width'), new_size.get('height'))
batch_size = Config_classification.get('batch_size')


In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        "/content/Codes/datasets/tmp/", validation_split=0.2, subset="training", seed=1337, image_size=image_size,
        batch_size=batch_size, shuffle=True, labels='inferred'
    )

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        "/content/Codes/datasets/tmp/", validation_split=0.2, subset="validation", seed=1337, image_size=image_size,
        batch_size=batch_size, shuffle=True,  labels='inferred'
    )

print(train_ds.class_names)

Found 270 files belonging to 2 classes.
Using 216 files for training.
Found 270 files belonging to 2 classes.
Using 54 files for validation.
['fire', 'nofire']


In [ ]:


def make_model_keras(input_shape, num_classes):
    """
    This function define the DNN Model based on the Keras example.
    :param input_shape: The requested size of the image
    :param num_classes: In this classification problem, there are two classes: 1) Fire and 2) Non_Fire.
    :return: The built model is returned
    """
    inputs = keras.Input(shape=input_shape)
    # x = data_augmentation(inputs)  # 1) First option
    x = inputs  # 2) Second option

    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    # x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.Conv2D(8, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x

    # for size in [128, 256, 512, 728]:
    for size in [8]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        residual = layers.Conv2D(size, 1, strides=2, padding="same")(previous_block_activation)

        x = layers.add([x, residual])
        previous_block_activation = x
    x = layers.SeparableConv2D(8, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs, name="model_fire")

In [ ]:
model = make_model_keras(input_shape=image_size + (3,), num_classes=2)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(1e-3), loss="binary_crossentropy", metrics=["accuracy"], )

import time
start_time = time.time()

res_fire = model.fit(train_ds, epochs=epochs, validation_data=val_ds, batch_size=batch_size)

time_A = print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/40
7/7 [==============================] - 18s 1s/step - loss: 0.0783 - accuracy: 0.9732 - val_loss: 0.4123 - val_accuracy: 0.9444
Epoch 2/40
7/7 [==============================] - 17s 1s/step - loss: 0.1487 - accuracy: 0.9178 - val_loss: 0.0264 - val_accuracy: 1.0000
Epoch 3/40
7/7 [==============================] - 17s 1s/step - loss: 0.0647 - accuracy: 0.9748 - val_loss: 0.5596 - val_accuracy: 0.7778
Epoch 4/40
7/7 [==============================] - 17s 1s/step - loss: 0.1327 - accuracy: 0.9184 - val_loss: 0.0660 - val_accuracy: 0.9815
Epoch 5/40
7/7 [==============================] - 17s 1s/step - loss: 0.0891 - accuracy: 0.9491 - val_loss: 1.2945 - val_accuracy: 0.7593
Epoch 6/40
7/7 [==============================] - 17s 1s/step - loss: 0.1075 - accuracy: 0.9438 - val_loss: 0.1193 - val_accuracy: 0.9630
Epoch 7/40
7/7 [==============================] - 17s 1s/step - loss: 0.1113 - accuracy: 0.9424 - val_loss: 2.5042 - val_accuracy: 0.6667
Epoch 8/40
7/7 [==================

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        "/content/Codes/datasets/tmp/", validation_split=0.2, subset="training", seed=1337, image_size=image_size,
        batch_size=batch_size, shuffle=True, labels='inferred', label_mode = "int"
    )



Found 270 files belonging to 2 classes.
Using 216 files for training.


In [ ]:
test = tf.keras.estimator.model_to_estimator(model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp7_ngl2vp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
hvd_model = hvd.KerasModel(model=model)
hvd_model = hvd_model.getModel()

start_time = time.time()

hvd_model.fit(train_ds, epochs=epochs, validation_data=val_ds, batch_size=batch_size)

time_B = print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/40
7/7 [==============================] - 17s 1s/step - loss: 0.1517 - accuracy: 0.9306 - val_loss: 0.6861 - val_accuracy: 0.7593
Epoch 2/40
7/7 [==============================] - 18s 1s/step - loss: 0.1415 - accuracy: 0.9491 - val_loss: 1.7217 - val_accuracy: 0.6667
Epoch 3/40
7/7 [==============================] - 17s 1s/step - loss: 0.1312 - accuracy: 0.9444 - val_loss: 1.8127 - val_accuracy: 0.6667
Epoch 4/40
7/7 [==============================] - 17s 1s/step - loss: 0.1420 - accuracy: 0.9398 - val_loss: 0.2282 - val_accuracy: 0.8704
Epoch 5/40
7/7 [==============================] - 17s 1s/step - loss: 0.1234 - accuracy: 0.9537 - val_loss: 1.8716 - val_accuracy: 0.6667
Epoch 6/40
7/7 [==============================] - 17s 1s/step - loss: 0.1486 - accuracy: 0.9491 - val_loss: 2.1071 - val_accuracy: 0.6667
Epoch 7/40
7/7 [==============================] - 17s 1s/step - loss: 0.1518 - accuracy: 0.9306 - val_loss: 2.6618 - val_accuracy: 0.6667
Epoch 8/40
7/7 [==================